In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
import os

#@markdown ## 1. ComfyUI 설치 
#@markdown ##### Install ComfyUI
#@markdown <details>
#@markdown   <summary>설치된 커스텀 노드 자세히 펼쳐보기 (details)</summary>
#@markdown
#@markdown **WD14-Tagger** 이미지->프롬프트
#@markdown > https://github.com/pythongosssss/ComfyUI-WD14-Tagger 
#@markdown
#@markdown **MTB Nodes** 이미지 / 애니메이션관련
#@markdown > https://github.com/melMass/comfy_mtb
#@markdown
#@markdown **rgthree** 편의성
#@markdown > https://github.com/rgthree/rgthree-comfy
#@markdown
#@markdown **was-node-suite** 편의성
#@markdown > https://github.com/WASasquatch/was-node-suite-comfyui
#@markdown
#@markdown **ComfyUI_ADV_CLIP_emb** A1111 프롬프트 가중치
#@markdown > https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb
#@markdown
#@markdown **ComfyUI-Easy-Use** Dynamic Prompt (Wildcards)
#@markdown > https://github.com/yolain/ComfyUI-Easy-Use
#@markdown
#@markdown **comfyui-lama-remover** Lama (remove object)
#@markdown > https://github.com/Layer-norm/comfyui-lama-remover
#@markdown
#@markdown **ComfyUI_essentials** Image, Mask
#@markdown > https://github.com/cubiq/ComfyUI_essentials
#@markdown
#@markdown </details>

HomePath = '/content'
Workspace = 'install_comfyui'

print('install...')
if not os.path.isdir(f'{HomePath}/{Workspace}'):
  with capture.capture_output() as cap:
    !wget https://huggingface.co/ninjaneural/webui/resolve/main/direct/install_comfyui_pack_20240516.tar -O install_comfyui.tar
    !tar -C {HomePath} -xvf install_comfyui.tar
    !rm install_comfyui.tar
    !wget https://raw.githubusercontent.com/ninjaneural/comfyui/master/misc/extra_model_paths.yaml -O {HomePath}/{Workspace}/extra_model_paths.yaml
    !sed -i -e "/    init_custom_nodes()/a\    folder_paths.add_model_folder_path('AnimateDiffEvolved_Models', '/content/animatediff/models')\n    folder_paths.add_model_folder_path('AnimateDiffMotion_LoRA', '/content/animatediff/motion_lora')" {HomePath}/{Workspace}/main.py
    %cd {HomePath}/{Workspace}

print("설치완료. 실행환경 설치을 진행해주세요")

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 2. 실행환경 설치
#@markdown ##### Install Env

%cd /content

!mkdir /content/models
!mkdir /content/models/checkpoints
!mkdir /content/models/controlnet_models
!mkdir /content/models/upscale_models
!mkdir /content/models/loras
!mkdir /content/models/vae
!mkdir /content/models/clip_vision
!mkdir /content/models/clip
!mkdir /content/models/unet

print('install library...')
with capture.capture_output() as cap:
  !pip install kornia
  !pip install torchsde
  !pip install insightface
  !pip install timm

  !pip install diffusers==0.25.0
  !pip install accelerate
  !pip install gradio==4.24.0
  !pip install av
  !pip install einops==0.7.0
  !pip install onnxruntime
  !pip install omegaconf>=2.0.0
  !pip install fvcore
  !pip install ultralytics
  !pip install rembg

  !pip install segment_anything
  !pip install addict
  !pip install yapf

  !apt install aria2

print("설치완료. 모델 다운로드를 진행해주세요")


In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 3. 모델 다운로드
#@markdown ##### Model Download

#@markdown **모델(checkpoint) URL (colab에 다운받는 모델)**
Checkpoint_Url = 'https://civitai.com/api/download/models/45601?type=Model&format=SafeTensor&size=full&fp=fp16' #@param {type:"string"}
#@markdown **모델 파일명**
Checkpoint_Filename = 'aurora_v10.safetensors' #@param {type:"string"}
#@markdown **모델 다운로드**
Checkpoint_Download = True #@param {type:"boolean"}

#@markdown -----
#@markdown *컨트롤넷 모델*

#@markdown **ControlNet 기본 모델 다운로드**
#@markdown > *openpose, lineart, softedge, depth, canny, tile, ip2p*  
ControlNet = True #@param {type:"boolean"}

#@markdown **ControlNet 추가 모델 다운로드**
#@markdown > *segment, inpaint, scribble, mlsd, normalbae, suffle*  
ControlNet_Extra = False #@param {type:"boolean"}

#@markdown **T2I Adapter 모델 다운로드**
#@markdown > *style, color, sketch, zoedepth, recolor*  
ControlNet_T2I = False #@param {type:"boolean"}

#@markdown -----
#@markdown *커스텀 노드 모델*

#@markdown **AnimateDiff 모델 다운로드**
AnimateDiff_Evolved = False #@param {type:"boolean"}

#@markdown **IPAdapter plus 모델 다운로드**
IPAdapter_plus = False #@param {type:"boolean"}

#@markdown **SVD 모델 다운로드**
SVD_Checkpoint = False #@param {type:"boolean"}

#@markdown **insightface 설치 (reactor 노드사용)**
Insightface = False #@param {type:"boolean"}

#@markdown **ic-light 설치**
ICLight = False

print('clip download...')
with capture.capture_output() as cap:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/models/clip_vision -o model.safetensors
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -d /content/models/clip_vision/ -o clip_vit14.bin
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/resolve/main/open_clip_pytorch_model.bin -d /content/models/clip -o open_clip_pytorch_model.bin

print('controlnet download...')
with capture.capture_output() as cap:

  ControlNet_Model_Path = "/content/models/controlnet_models"
  if ControlNet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_ip2p_fp16.safetensors

  if ControlNet_Extra:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_seg_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_inpaint_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_scribble_fp16.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_mlsd_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_normalbae_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_shuffle_fp16.safetensors

  if ControlNet_T2I:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_style_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_color_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d {ControlNet_Model_Path} -o t2iadapter_sketch_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d .{ControlNet_Model_Path} -o t2iadapter_zoedepth_sd15v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d {ControlNet_Model_Path} -o ioclab_sd15_recolor.safetensors  

print('animatediff download...')
with capture.capture_output() as cap:

  if AnimateDiff_Evolved:
    !mkdir /content/animatediff
    !mkdir /content/animatediff/models
    !mkdir /content/animatediff/motion_lora
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d /content/animatediff/models -o mm_sd_v15_v2.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -d /content/animatediff/models -o mm_sd_v15.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -d /content/animatediff/models -o mm_sd_v14.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt -d /content/animatediff/models -o v3_sd15_mm.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.ckpt -d /content/animatediff/models -o temporaldiff-v1-animatediff.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanLeft.ckpt -d /content/animatediff/motion_lora -o v2_lora_PanLeft.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanRight.ckpt -d /content/animatediff/motion_lora -o v2_lora_PanRight.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt -d /content/animatediff/motion_lora -o v2_lora_ZoomOut.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt -d /content/animatediff/motion_lora -o v2_lora_ZoomIn.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltUp.ckpt -d /content/animatediff/motion_lora -o v2_lora_TiltUp.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltDown.ckpt -d /content/animatediff/motion_lora -o v2_lora_TiltDown.ckpt
    
print('ipadapter download...')
with capture.capture_output() as cap:

  if IPAdapter_plus:
    !mkdir /content/ipadapter
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -d /content/ipadapter -o ip-adapter_sd15.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.safetensors -d /content/ipadapter -o ip-adapter_sd15_light.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -d /content/ipadapter -o ip-adapter-plus_sd15.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -d /content/ipadapter -o ip-adapter-plus-face_sd15.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -d /content/ipadapter -o ip-adapter-full-face_sd15.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin -d /content/ipadapter -o ip-adapter-faceid_sd15.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin -d /content/ipadapter -o ip-adapter-faceid-plus_sd15.bin
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin -d /content/ipadapter -o ip-adapter-faceid-plusv2_sd15.bin

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors -d /content/models/loras -o ip-adapter-faceid-plusv2_sd15_lora.safetensors
    
# svd checkpoint
print('svd download...')
with capture.capture_output() as cap:
  if SVD_Checkpoint:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid/resolve/main/svd.safetensors -d /content/models/checkpoints -o svd.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors -d /content/models/checkpoints -o svd_xt.safetensors

  if Insightface:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth -d {HomePath}/{Workspace}/models/facerestore_models -o GFPGANv1.4.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ninjaneural/webui/resolve/main/inswapper_128.onnx -d {HomePath}/{Workspace}/models/insightface -o inswapper_128.onnx
    !pip install insightface

# checkpoint
print('checkpoint download...')
with capture.capture_output() as cap:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d /content/models/checkpoints -o {Checkpoint_Filename}

  # upscaler
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/models/upscale_models -o 4x-UltraSharp.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gemasai/4x_NMKD-Siax_200k/resolve/main/4x_NMKD-Siax_200k.pth -d /content/models/upscale_models -o 4x_NMKD-Siax_200k.pth
  
  # vae
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/models/vae -o vae-ft-mse-840000-ema-pruned.vae.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/models/vae -o kl-f8-anime2.ckpt

  # lora
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/models/loras -o add_detail.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors" -d /content/models/loras -o lcm-lora-sd15.safetensors

print('unet download...')
with capture.capture_output() as cap:
  if ICLight:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/lllyasviel/ic-light/resolve/main/iclight_sd15_fc.safetensors" -d /content/models/unet -o iclight_sd15_fc.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/lllyasviel/ic-light/resolve/main/iclight_sd15_fbc.safetensors" -d /content/models/unet -o iclight_sd15_fbc.safetensors

%cd {HomePath}/{Workspace}

print("다운로드완료. ComfyUI 실행을 진행해주세요")

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## (옵션) 추가 커스텀 노드 설치
#@markdown ##### (Optional) Install Custom Nodes
#@markdown * Gemini API키
GEMINI_API_KEY = '' #@param {type:"string"}

#@markdown * ReActor
ReActor = True #@param {type:"boolean"}

#@markdown * DreamTalk
DREAMTALK = False #@param {type:"boolean"}

#@markdown * IDM-VTON
IDM_VTON = False #@param {type:"boolean"}

#@markdown <br/>
#@markdown <details>
#@markdown   <summary>추가 커스텀노드 자세히 펼쳐보기 (details)</summary>
#@markdown
#@markdown **comfyui_segment_anything** Segment Anything, DINO
#@markdown > https://github.com/storyicon/comfyui_segment_anything
#@markdown
#@markdown **ComfyUI-Video-Matting** 배경 분리 Robust Video Matting (RVM)
#@markdown > https://github.com/Fannovel16/ComfyUI-Video-Matting
#@markdown
#@markdown **ComfyUI-Gemini** Gemini 연동
#@markdown > https://github.com/ZHO-ZHO-ZHO/ComfyUI-Gemini
#@markdown
#@markdown </details>

%cd {HomePath}/{Workspace}/custom_nodes

print('custom nodes download...')
with capture.capture_output() as cap:
  !git clone https://github.com/storyicon/comfyui_segment_anything
  !git clone https://github.com/Fannovel16/ComfyUI-Video-Matting

  if DREAMTALK:
    !git clone https://github.com/neuralninja22/ComfyUI_DDreamtalk
    %cd ComfyUI_DDreamtalk
    !pip install -r requirements.txt
    %cd ..
  
  if IDM_VTON:
    !git clone https://github.com/neuralninja22/IDM-VTON
    !apt install aria2

    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/openpose/ckpts/body_pose_model.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/levihsu/OOTDiffusion/resolve/main/checkpoints/openpose/ckpts/body_pose_model.pth -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/openpose/ckpts -o body_pose_model.pth
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing/parsing_atr.onnx
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/humanparsing/parsing_atr.onnx -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing -o parsing_atr.onnx
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing/parsing_lip.onnx
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/humanparsing/parsing_lip.onnx -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/humanparsing -o parsing_lip.onnx
    !rm /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/densepose/model_final_162be9.pkl
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yisol/IDM-VTON/resolve/main/densepose/model_final_162be9.pkl -d /content/install_comfyui/custom_nodes/IDM-VTON/ckpt/densepose -o model_final_162be9.pkl
    
  if not ReActor:
    !mv comfyui-reactor-node ../../

  if GEMINI_API_KEY:
    !git clone https://github.com/ZHO-ZHO-ZHO/ComfyUI-Gemini
    !sed -i -e 's/"api_key": "[^"]*"/"api_key": "{GEMINI_API_KEY}"/' ./ComfyUI-Gemini/config.json

  if ICLight:
    !git clone https://github.com/kijai/ComfyUI-IC-Light

%cd {HomePath}/{Workspace}

In [ ]:
from IPython.utils import capture
from IPython.display import clear_output

#@markdown ## 4. ComfyUI 실행
#@markdown ##### Run ComfyUI

#@markdown -----
#@markdown *터널링*

#@markdown **Localtunnel**
LocalTunnel = True #@param {type:"boolean"}

#@markdown **임시서버 사용** (LocalTunnel가 동작하지 않을때 이용해주세요)
TunnelTempServer = True #@param {type:"boolean"}

%cd {HomePath}/{Workspace}

if LocalTunnel:
  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()

    command = ["lt", "--port", "{}".format(port)]
    if TunnelTempServer:
      command = ["lt", "--host", "http://lt.dabayo.com", "--port", "{}".format(port)]
    else:
      print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    def run_process(count_down):
      p = subprocess.Popen(command, stdout=subprocess.PIPE)
      for line in p.stdout:
        url_output = line.decode()
        your_domain = url_output[13:]
        try:
          response = urllib.request.urlopen(your_domain, timeout=15)
          print("your url : " + your_domain)
          data = response.read().decode('utf8').strip("\n")
        except Exception as e:
          print(f"General Error: {e}")
          print("> retry")
          p.kill()
          time.sleep(3)
          if count_down > 0:
            run_process(count_down-1)

    run_process(3)

  threading.Thread(target=iframe_thread, daemon=True, args=(18188,)).start()

!python main.py --port 18188 --dont-print-server